In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import csv

# 1. 다음카페 로그인 페이지 열기

In [10]:
driver = webdriver.Chrome()
# 설치 폴더에 주의합니다. 

driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
# 19년 5월부터 로그인 페이지 주소가 살짝 바뀌었네요. 

time.sleep(3)
# 페이지 전환시에는 적당한 시간을 줍니다. 
# 1. 과도한 크롤링 방지.
# 2. 페이지 전환이 완료되기 전에 다음 명령 실행되는 것 방지.
#    AJAX를 사용한 페이지는 페이지 전환시 딜레이가 꼭 필요한 경우도 있습니다.

# 2. 아이디, 패스워드 수동 입력

# 3. 카페 이름과 주소 지정

In [11]:
CAFE_NAME = 'rocksoccer' 
# 까페 이름을 넣어준다. 예제는 이종격투기... 

BOARD_NAME = 'ADs2' 
# 게시판 주소의 마지막 4자리(?)를 넣어준다.
# http://m.cafe.daum.net/ssaumjil/Jntr
QUERY = '쿠팡'

# 4. 락싸 국내토크방 게시판 이동 후 QUERY 검색

In [12]:
driver.get(f'http://m.cafe.daum.net/{CAFE_NAME}/{BOARD_NAME}/search?r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&query={QUERY}')
time.sleep(3)

# 이제 위에서 지정한 까페의 게시판으로 이동합니다. 
# 바로 게시물로 이동해도 됩니만.. 
# 계정정지의 압봵이 있으니... 게시판 리스트를 보고 게시물 하나만 소박하게 스크래핑하는 식으로... 

In [40]:
# 마지막 페이지까지 클릭
def go_to_last_page():
    wait = WebDriverWait(driver, 10)
    while True:
        try:
            # '다음 페이지' 버튼을 찾기
            next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.btn_page.btn_next")))
            next_button_text = next_button.text.strip()
            # 버튼이 비활성화 상태인지 확인
            if next_button_text == '다음 목록이 없습니다.':
                print("Reached the last page.")
                break

            # '다음 페이지' 버튼 클릭
            next_button.click()
            # 로딩 시간을 기다리기 위해 sleep 사용
            time.sleep(5)
        except TimeoutException:
            print("Reached the last page.")
            break

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

# 마지막 페이지 번호 알아내기
def get_last_page():
    soup = go_to_last_page()
    page_list = []
    # 모든 'num_page' 클래스를 가진 span 태그 내에서 페이지 번호 추출
    for span in soup.find_all("span", class_="num_page"):
        # 'link_page' 클래스를 가진 태그(a 태그 또는 em 태그) 찾기
        page_tag = span.find(class_="link_page")
        if page_tag:
            # 페이지 번호를 리스트에 추가
            page_list.append(page_tag.get_text(strip=True))
    max_page = page_list[-1]
    print(f"총 {max_page}개의 페이지가 있습니다")
    return max_page

# 페이지 링크 모두 가져오기
def get_boards(page_num):
    boards=[]
    for page in range(page_num):
        boards.append(f"https://m.cafe.daum.net/{CAFE_NAME}/{BOARD_NAME}/search/all?query={QUERY}&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page={page+1}")
    return boards

# 게시글 링크 가져오기
def get_posts():
    global QUERY
    global PAGES
    board_links = get_boards(PAGES)
    posts = []
    for board_link in board_links:
        req = requests.get(board_link)
        print(req.status_code)
        soup = BeautifulSoup(req.text, 'html.parser')
        tds = soup.find_all('a', {'class':'link_cafe #search_result_list'})
        for td in tds:
            if td is not None:
               posts.append('m.cafe.daum.net'+td['href'])
    print(f"총 {len(posts)}개의 글 링크를 찾았습니다")
    
    # 게시글 링크 csv로 저장
    post_file = open(f"ROCKSOCCER_{QUERY}_{PAGES}pages_inner_links.csv", mode='w', encoding='utf-8')
    writer = csv.writer(post_file)
    for post in posts:
        writer.writerow([post])
    post_file.close()
    return posts

In [36]:
go_to_last_page()

Reached the last page.


<html class="os_windows chrome pc version_124_0_0_0" lang="ko"><head>
<title>樂soccer</title>
<meta content="user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, width=device-width, shrink-to-fit=no" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="telephone=no" name="format-detection"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/favicon.ico" rel="shortcut icon"/> <link href="//t1.daumcdn.net/cafe_image/mobile/ico_favicon_mw.png" rel="apple-touch-icon-precomposed"/>
<link href="//t1.daumcdn.net/cafe_cj/mobileweb/build/css/mobile_cafe-1f16e55e79.min.css" rel="stylesheet" type="text/css"/>
<link href="//t1.daumcdn.net/cafe_cj/mobileweb/build/css/custom-fa4bd1932b.min.css" rel="stylesheet" type="text/css"/>
<link href="//t1.daumcdn.net/cafe_cj/mobileweb/build/css/mobile_cafe_invite-e41670d9b5.min.css" rel="stylesheet" type="text/css"/>
<script>
		var CAFEAPP = {
			CONTEXT_PATH: "/rocksoccer

In [42]:
get_last_page()

Reached the last page.
총 32개의 페이지가 있습니다


'32'

In [43]:
get_boards(32)

['https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query=쿠팡&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page=1',
 'https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query=쿠팡&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page=2',
 'https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query=쿠팡&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page=3',
 'https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query=쿠팡&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page=4',
 'https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query=쿠팡&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page=5',
 'https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query=쿠팡&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page=6',
 'https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query=쿠팡&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page=7',
 'https://m.cafe.daum.net/rocksoccer/ADs2/search

In [44]:
# 크롤링할 페이지 수 설정
PAGES = int(get_last_page())
get_posts()

Reached the last page.
총 32개의 페이지가 있습니다
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
총 637개의 글 링크를 찾았습니다


['m.cafe.daum.net/rocksoccer/ADs2/489560?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/488894?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/488886?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/488874?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/487459?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/487425?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/487221?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/485759?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/485233?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/484590?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/484314?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/484256?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/483602?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/481478?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/480804?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/480763?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/480439?searchView=Y',
 'm.cafe.daum.net/rocksoccer/ADs2/479359?searchV

# 링크 csv파일을 로드하고 링크로 이동

In [48]:
def save_to_file():
    file = open(f'RockSoccer_{QUERY}_.csv', mode='a', encoding='utf-8')
    writer = csv.writer(file)
    writer.writerow(['title', 'user_id', 'post_time', 'post', 'view_cnt', 'reply_cnt', 'reply_author', 'reply_text', 'reply_time'])
    file.close()
    return file
def get_contents():
    global rocksoccer_results
    post_links = get_posts()
    for post_link in post_links:
        content = extract_info(post_link)
        append_to_file(f"RockSoccer_{QUERY}_{PAGES}.csv", content)
    return print("모든 작업이 완료되었습니다.")
driver.get(url) # 게시물의 주소로 이동합니다. 
time.sleep(2)

### 페이지 소스를 BeautifulSoup(이하 BS)에게 넘기는 것을 마지막으로 셀레니움의 역할은 끝이 납니다.

In [57]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser') 

# repr(soup)

### 제목 찾기

In [58]:
subject = soup.body.find('h3', class_='tit_subject')
# BS는 class명을 이용해서 h3을 바로 찾을 수 있습니다. 
# 클래스 명은 ID와 달리 중복이 가능합니다. 
# 태그와 클래스명까지 중복되어 겹치는 경우 첫번째 노드가 검색됩니다.  
# 다행히 이 HTML 문서에서 'tit_subject' 클래스는 한번만 사용됩니다. ^^ 
# 중복을 걱정할 필요가 없네요. 
# 중복 여부는 개발자 도구의 검색 기능을 이용해 보십시오

In [59]:
import pandas as pd

# DataFrame 초기화
df_columns = ['num', 'subject', 'nick', 'write_time', 'views', 'url', 'contents', 'cmt_num', 'cmt_writer', 'cmt_time', 'cmt_txt']
data_frame = pd.DataFrame(columns=df_columns)

# 게시물 정보 추가 (게시물 본문 크롤링 부분에 추가)
if subject is None:
    print(url, '지워진 게시물입니다.')
else:
    subject = subject.get_text(strip=True)
    if soup.body.find('span', class_='txt_subject').find('span', class_='sr_only').get_text() == '작성자':
        nick = soup.body.find('span', class_='sr_only').next_sibling
    else:
        nick = ''
    num_subject = soup.body.find_all('span', class_='num_subject')
    write_time = num_subject[0].get_text()
    views = num_subject[1].get_text()
    contents = soup.body.find('div', id='article').get_text('\n', strip=True)
    
    # 게시물 정보를 DataFrame에 추가
    data_frame = data_frame.append({'num': inp_num, 'subject': subject, 'nick': nick, 'write_time': write_time, 'views': views, 'url': url, 'contents': contents}, ignore_index=True)

# 댓글 정보 추가 (댓글 크롤링 부분에 추가)
for i in range(cmt_page_max):
    j = cmt_page_max-i
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    cmt_all = soup.body.find('ul' ,id="commentList").find_all('li')
    
    for k in cmt_all:
        if k.div.span.find_all('span', class_="txt_bar"):
            if k['class'] == ['reply_on']:
                cmt_reply = True
                reply_prefix = 'ㄴ'
            else:
                cmt_reply = False
                reply_prefix = ''
                
        cmt_num = inp_num +'_'+ k['id'].split('comment_')[1]
        cmt_writer = k.div.find('span', class_='username').get_text(strip=True)
        cmt_time = k.div.span.find('span', class_="created_at").get_text()
        cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
        
        # 댓글 정보를 DataFrame에 추가
        data_frame = data_frame.append({'cmt_num': reply_prefix + cmt_num, 'cmt_writer': cmt_writer, 'cmt_time': cmt_time, 'cmt_txt': cmt_txt}, ignore_index=True)
    if j > 1:
        driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments?prev_page={3}&mode=regular&cdepth={4}&page={5}'.format(CAFE_NAME,BOARD_NAME,inp_num,j,'0002100000',(j-1)))
        time.sleep(2)


489567 [☆서울]죽으라든지 큰 부상 당해서 나오지 말라는 등 격한 메시지를 받으니까 선수가 심리적으로 흔들리는 부분이 있었다 킹덤 왕기 2시간 4분 전 605 http://m.cafe.daum.net/rocksoccer/ADs2/489567
https://naver.me/GABWq0II
사건의 중심에 선 백종범은 경인더비 이후 일부 팬들로부터 수위 높은 메시지를 받았다. 대구와의 경기 전 취재진과 인터뷰를 진행한 김기동 감독은 “죽으라든지 큰 부상 당해서 나오지 말라는 등 격한 메시지를 받으니까 선수가 심리적으로 흔들리는 부분이 있었다”라고 말하면서 백종범을 걱정했다.
어제 메세지 내용은 기사에서 못봤었는데 ㅡㅡ
진짜 참담하네요
댓글 개수 : 9
cmt_page_max 1
489567_1 [ Sososo ] 17:51 네? ㅜㅜㅜ
489567_2 [ 초록초롱초롬 ] 17:57 인간 이하의 것들이 많네요 참...
489567_3 [ 박지현 ] 18:02 에라이 ㅋㅋㅋ 봐 봐 정신 못 차렸다니까... 처벌이 두려워 가면 쓰는 거지 벌거지들!
489567_4 [ 지동원 ] 18:15 미친건가 진짜..
489567_5 [ 본투비블루 ] 18:16 사회가 너무 병들었다... 본인들 하류인생 화풀이 대상이 필요한듯
489567_6 [ Tensegrity Universe and Philosophy ] 18:18 전부 법적대응 하면 안되려나.. 백종범 만의 문제는 아닐텐데..ㅠㅠㅠ
489567_8 [ 심재원 송종국 ] 18:33 저 메시지를 서울 인천 외 다른 팀 팬들이 보냈을리는 없고...무엇보다 물병 투척 경기 직후 앵간한 축구 사이트, 그 경기 관련 영상 댓글에 많았던 반응이 '백종범 키퍼가 도발해서' 라는 내용이었는데그럼 뭐 누가 보냈는지는...
489567_9 [ 수원삼성조유리 ] 18:40 선수들 저런거 많이 받을거라고 생각함괜히 인스타 닫고 그러는게 아닌쓰레기들
489567_10 [ 키나페 ] 19:23 지인 중 프로선수가 몇 있는게경기 끝나고 욕 디엠이나